In [1]:
import json
import pandas as pd
import statistics

spiList = [5.55, 5.72, 7.3, 7.75, 8.4, 9, 8.8, 8.2]
print(statistics.mean(spiList))

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import operator

from tqdm import tqdm

7.59


In [9]:
from gensim.models.keyedvectors import FastTextKeyedVectors
fasttext = FastTextKeyedVectors.load("D:/fasttext_word2vec/araneum_none_fasttextcbow_300_5_2018/araneum_none_fasttextcbow_300_5_2018.model")


C:\Users\Asus\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [ ]:
НЕПОНЯТНЫЕ ПРБЛЕМЫ С ВЕКТОРОМ ЧАСТОТНОСТЕЙ

In [3]:
with open ("lyashevskaya_freq_dict.json" , "r", encoding="utf-8") as f:
    lyashevskaya_freq_dict = json.load(f)
    
with open ("C:\Autotutor\improved_approach\colloc\music_unigr_freq.json", "r", encoding='utf-8') as f:
    unigramm_db = json.load(f)
    
    
with open ("D:\input\music_smart_colloc_freq.json", "r", encoding='utf-8') as f:
    colloc_db = json.load(f)
    
words_indexed_dict = []
with open("big_musician_colloc_lit.txt", "r", encoding = "utf-8") as f:
    for line in f.readlines():
        words_indexed_dict.append(line[:-1])

In [4]:
words_indexed_dict[0]

'оказываться'

In [5]:
USER_NAME = 'big_musician'

In [6]:
marked_word_db = pd.read_csv(USER_NAME +  "_word_db.csv",header = None)

In [7]:
def get_vector(ngramm):
    ngramm_lemmas_list = ngramm.split()
    freq_list = []
    local_corpora_freq_list = []
    w2v_list = []
    for lemma in ngramm_lemmas_list:
        
        try:
            w2v = fasttext[lemma]
            w2v_list.append(w2v)
        except:
            pass
        """
        print(lemma)
        w2v = fasttext[lemma]
        w2v_list.append(w2v)
        """
        
        if lemma in lyashevskaya_freq_dict:
            freq_list.append(lyashevskaya_freq_dict[lemma])
        else:
            freq_list.append(0)
            
        if lemma in unigramm_db:
            local_corpora_freq_list.append(unigramm_db[lemma])
        else:
            local_corpora_freq_list.append(0)
    colleted_w2v_count = len(w2v_list)
    if colleted_w2v_count > 0:
        vect_sum = 300 * [0]
        for w2v in w2v_list:
            vect_sum += w2v
        vect_sum /=  colleted_w2v_count
    else:
        print(ngramm_lemmas_list, "none in fasttext")
        vect_sum = None
    if len(local_corpora_freq_list) > 0:
        local_corpora_mean = statistics.mean(local_corpora_freq_list)
    else:
        local_corpora_mean = None
    if len(freq_list) > 0:
        freq_mean = statistics.mean(freq_list)
    else:
        freq_mean = None
    
    #return (local_corpora_mean,freq_mean, vect_sum)#????
    return (freq_mean,local_corpora_mean, vect_sum)

In [11]:
list_of_all_words_similarity = {}
with open("./colloc/music_unigr_freq.json", 'r', encoding = 'utf-8') as f:
    music_unigr_db = json.load(f)
    all_words = list(music_unigr_db.keys())
    for colloc_word_ind in tqdm(range(len(all_words))):
        if not all_words[colloc_word_ind].isdigit():
            current_word_similarity = []
            #print(colloc[1][0])
            colloc_word_vector = get_vector(all_words[colloc_word_ind])
            if np.any(colloc_word_vector[0]) and np.any(colloc_word_vector[1]) and np.any(colloc_word_vector[2]):
                freq_vec = np.array([colloc_word_vector[0], colloc_word_vector[1]]).reshape(1,-1)
                #print("initial word freq_vec", colloc_word, freq_vec)
                colloc_w2v = np.array(colloc_word_vector[2]).reshape(1,-1)
                absolute_equality_found = False
                for db_word_index in range(len(marked_word_db)):
                    #print(words_indexed_dict[db_word_index])
                    compared_raw_word_vect = marked_word_db.iloc[db_word_index][:-1]

                    comp_word2vec = compared_raw_word_vect[2:]

                    #print(db_word_index, len(comp_word2vec))
                    comp_colloc_w2v = np.array(comp_word2vec).reshape(1,-1)
                    if np.any(comp_word2vec) and np.any(colloc_w2v):
                        w2v_sim = cosine_similarity(colloc_w2v,comp_colloc_w2v)
                        #print("w2v", sim)
                    else:
                        w2v_sim = [[0]]

                    comp_freq_vec = np.array([compared_raw_word_vect[1], compared_raw_word_vect[0]]).reshape(1,-1)
                    sim_freq = cosine_similarity(freq_vec,comp_freq_vec)
                    #print("freq", sim_freq)
                    #print(comp_freq_vec)
                    w2v_sim = w2v_sim[0][0]
                    sim_freq = sim_freq[0][0]

                    average_similarity = w2v_sim*0.8 + sim_freq*0.2
                    if average_similarity >= 1:
                        absolute_equality_found = True

                    word_comp_json = {"marked_colloc_db_index":db_word_index, "sim_colloc":words_indexed_dict[db_word_index],"w2v_sim":w2v_sim,"freqence_similarity":sim_freq}
                    if average_similarity> 0.6 and w2v_sim > 0.65 and absolute_equality_found == False:
                        current_word_similarity.append((word_comp_json,average_similarity))
                    elif average_similarity >= 0.75 and absolute_equality_found == True:
                        current_word_similarity.append((word_comp_json,average_similarity))
                current_word_similarity.sort(key = operator.itemgetter(1), reverse= True)
                if absolute_equality_found == True:
                    only_absolute_equalitie = []
                    for simil_el in current_word_similarity:
                        if simil_el[1] >= 0.85:
                            only_absolute_equalitie.append(simil_el)
                    list_of_all_words_similarity[all_words[colloc_word_ind]] = only_absolute_equalitie[:5]
                else:
                    list_of_all_words_similarity[all_words[colloc_word_ind]] = current_word_similarity[:5]
            
       


  0%|                                                                             | 12/56248 [00:06<8:40:43,  1.80it/s]

['о'] none in fasttext


  0%|                                                                             | 26/56248 [00:15<9:21:58,  1.67it/s]

['и'] none in fasttext


  0%|                                                                             | 40/56248 [00:23<9:12:00,  1.70it/s]

...

 97%|███████████████████████████████████████████████████████████████████████▋  | 54468/56248 [3:16:51<06:26,  4.61it/s]

['🙏🏾🗣'] none in fasttext
['t🎤'] none in fasttext


 97%|████████████████████████████████████████████████████████████████████████  | 54733/56248 [3:17:17<05:27,  4.62it/s]

['i8'] none in fasttext


100%|██████████████████████████████████████████████████████████████████████████| 56248/56248 [3:20:47<00:00,  4.67it/s]


In [12]:
with open ("big_musician_unigr_colloc_similairities.json", 'w') as f:
    json.dump(list_of_all_words_similarity, f , indent = 4, ensure_ascii=False)